# Лабораторна робота 4 (2 частина)

Романенко Ілля ФІТ 4-7

23 варіант

In [197]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

## Крок 1

Завантажуємо датасет та проведемо аналіз даних

In [198]:
filePath = './titanic_2.csv'

dataFrame = pd.read_csv(filePath)

dataFrame.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [199]:
print('кількість колонок: ', dataFrame.shape[1])
print('кількість рядків: ', dataFrame.shape[0])

print(dataFrame.isnull().sum())

кількість колонок:  12
кількість рядків:  418
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [200]:
print('типи даних по колонках:\n',  dataFrame.dtypes)

типи даних по колонках:
 PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


видалимо стовпчик Cabin, так як він має занадто багато пропущених даних

In [201]:
dataFrame = dataFrame.drop(['Cabin'], axis=1)

для всіх інших пропущених значень в клітинках (в основному для стовпчика Age) заповнимо їх середнім медіанним значенням

In [202]:
dataFrame = dataFrame.fillna(dataFrame.mean(numeric_only=True), axis = 0)

перевіримо на порожні значення

In [203]:
dataFrame.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

перевіримо дані на дуплікати

In [204]:
dataFrame.duplicated().sum()

0

видалимо непотрібні колонки

In [205]:
dataFrame = dataFrame.drop(['PassengerId', 'SibSp', 'Parch', 'Name', 'Ticket', 'Embarked'], axis=1)

замінимо текстові дані про стать, числами

In [206]:
print(dataFrame['Sex'].unique())

['male' 'female']


In [207]:
dataFrame['Sex'] = dataFrame['Sex'].replace({'female': 1, 'male': 0}).astype(int)

dataFrame.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,0,34.5,7.8292
1,1,3,1,47.0,7.0000
2,0,2,0,62.0,9.6875
3,0,3,0,27.0,8.6625
4,1,3,1,22.0,12.2875


In [208]:
dataFrame.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
Fare        0
dtype: int64

In [209]:
print('типи даних по колонках:\n',  dataFrame.dtypes)

типи даних по колонках:
 Survived      int64
Pclass        int64
Sex           int64
Age         float64
Fare        float64
dtype: object


In [210]:
dataFrame.describe()

,Survived,Pclass,Sex,Age,Fare
count,418.000000,418.000000,418.000000,418.000000,418.000000
mean,0.363636,2.265550,0.363636,30.272590,35.627188
std,0.481622,0.841838,0.481622,12.634534,55.840500
min,0.000000,1.000000,0.000000,0.170000,0.000000
25%,0.000000,1.000000,0.000000,23.000000,7.895800
50%,0.000000,3.000000,0.000000,30.272590,14.454200
75%,1.000000,3.000000,1.000000,35.750000,31.500000
max,1.000000,3.000000,1.000000,76.000000,512.329200


In [211]:
dataFrameCopy = dataFrame.copy() # копіюємо датасет для повторної обробки

## Крок 2

побудова моделей

розділяємо дані на навчальний та тестовий набір, та масштабуємо дані

In [212]:
y = dataFrame['Survived']
dataFrame = dataFrame.drop(['Survived'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(dataFrame, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

dataFrame = scaler.fit_transform(dataFrame)

### Побудуємо три моделі класифікації та підберемо параметри моделі за допомогою grid search

##### логістична регресія

In [213]:
logReg = LogisticRegression(max_iter=2000, solver='liblinear')
logRegParams = {
    'C': [0.001, 0.01, 0.05, 0.09, 0.1, 0.11, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2']
}

logRegGrid = GridSearchCV(logReg, logRegParams, cv=5)
logRegGrid.fit(X_train, y_train)
logRegBest = logRegGrid.best_estimator_

print(f'найкращі параметри: {logRegGrid.best_params_}\n')

y_pred = logRegBest.predict(X_test)
acuracy = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'точність: {acuracy}\n mse: {mse}\n r2: {r2}')

найкращі параметри: {'C': 0.01, 'penalty': 'l1'}

точність: 1.0
 mse: 0.0
 r2: 1.0


#### дерево рішень

In [214]:
tree = DecisionTreeClassifier()
treeParams = {
    'max_depth': [1, 2, 3, 5],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1, 2, 3, 5]
}

treeGrid = GridSearchCV(tree, treeParams, cv=5)
treeGrid.fit(X_train, y_train)
treeBest = treeGrid.best_estimator_

print(f'найкращі параметри: {treeGrid.best_params_}\n')

y_pred = treeBest.predict(X_test)
acuracy = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'точність: {acuracy}\n mse: {mse}\n r2: {r2}')

найкращі параметри: {'max_depth': 1, 'min_samples_leaf': 1, 'min_samples_split': 2}

точність: 1.0
 mse: 0.0
 r2: 1.0


#### випадковий ліс

In [215]:
randFrst = RandomForestClassifier()
randFrstParams = {
    'n_estimators': [10, 50, 100],
    'max_depth': [1, 2, 5]
}

randFrstGrid = GridSearchCV(randFrst, randFrstParams, cv=5)
randFrstGrid.fit(X_train, y_train)
randFrstBest = randFrstGrid.best_estimator_

print(f'найкращі параметри: {randFrstGrid.best_params_}\n')

y_pred = randFrstBest.predict(X_test)
acuracy = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'точність: {acuracy}\n mse: {mse}\n r2: {r2}')

найкращі параметри: {'max_depth': 1, 'n_estimators': 10}

точність: 1.0
 mse: 0.0
 r2: 1.0


Як бачимо, так як в цьому датасеті є 100% кореляція між статтю пасажира та його відсотком виживання, то будь яка модель може показати гарний результат

## Крок 3

виведемо найкращі параметри

In [216]:
bestParams = {
    'model' : ['LogisticRegression', 'DecisionTreeClassifier', 'RandomForestClassifier'],
    'best params': [
        logRegGrid.best_params_,
        treeGrid.best_params_,
        randFrstGrid.best_params_
    ]
}

bestParamsDataFrame = pd.DataFrame(bestParams)

bestParamsDataFrame

,model,best params
0,LogisticRegression,"{'C': 0.01, 'penalty': 'l1'}"
1,DecisionTreeClassifier,"{'max_depth': 1, 'min_samples_leaf': 1, 'min_s..."
2,RandomForestClassifier,"{'max_depth': 1, 'n_estimators': 10}"


виведемо звіт про класифікацію моделі

In [217]:
models = {
    'Logarithmic Regression': logRegBest,
    'Decision Tree': treeBest,
    'Random Forest': randFrstBest
}

for model_name, model in models.items():
    print(f"модель: {model_name}")
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
    print('-' * 60)

модель: Logarithmic Regression
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        34

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84

------------------------------------------------------------
модель: Decision Tree
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        34

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84

------------------------------------------------------------
модель: Random Forest
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1

## Крок 4

ми візьмемо логарифмічну модель. Хоча це не має принципового значення в даному випадку, яку модель обирати

In [218]:
y_test_reset = y_test.reset_index(drop=True)

logReg2 = LogisticRegression(**logRegGrid.best_params_, max_iter=2000, solver='liblinear')
logReg2.fit(X_train, y_train)

np.random.seed(42)
randomIndices = np.random.choice(X_test.shape[0], 10, replace=False)
X_random = X_test[randomIndices]
y_random_actual = y_test_reset[randomIndices]

y_random_pred = logReg2.predict(X_random)

comparisonDataFrame = pd.DataFrame({
    'Sample Index': randomIndices,
    'Real Class': y_random_actual,
    'Predicted Class': y_random_pred
})

comparisonDataFrame

,Sample Index,Real Class,Predicted Class
73,73,1,1
0,0,0,0
58,58,1,1
22,22,1,1
12,12,1,1
70,70,0,0
10,10,0,0
18,18,0,0
4,4,1,1
65,65,0,0


## Крок 5

спробуємо створити модель без даних про стать

In [219]:
dataFrame = dataFrameCopy.copy()
dataFrame = dataFrame.drop(['Sex'], axis=1)

dataFrame.head()

,Survived,Pclass,Age,Fare
0,0,3,34.5,7.8292
1,1,3,47.0,7.0000
2,0,2,62.0,9.6875
3,0,3,27.0,8.6625
4,1,3,22.0,12.2875


In [220]:
y = dataFrame['Survived']
dataFrame = dataFrame.drop(['Survived'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(dataFrame, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

dataFrame = scaler.fit_transform(dataFrame)

In [221]:
logReg = LogisticRegression(max_iter=2000, solver='liblinear')
logRegParams = {
    'C': [0.001, 0.01, 0.05, 0.09, 0.1, 0.11, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2']
}

logRegGrid = GridSearchCV(logReg, logRegParams, cv=5)
logRegGrid.fit(X_train, y_train)
logRegBest = logRegGrid.best_estimator_

print(f'найкращі параметри: {logRegGrid.best_params_}\n')

y_pred = logRegBest.predict(X_test)
acuracy = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'точність: {acuracy}\n mse: {mse}\n r2: {r2}')

найкращі параметри: {'C': 1, 'penalty': 'l1'}

точність: 0.6071428571428571
 mse: 0.39285714285714285
 r2: -0.6305882352941174


In [222]:
randFrst = RandomForestClassifier()
randFrstParams = {
    'n_estimators': [10, 50, 100],
    'max_depth': [1, 2, 5]
}

randFrstGrid = GridSearchCV(randFrst, randFrstParams, cv=5)
randFrstGrid.fit(X_train, y_train)
randFrstBest = randFrstGrid.best_estimator_

print(f'найкращі параметри: {randFrstGrid.best_params_}\n')

y_pred = randFrstBest.predict(X_test)
acuracy = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'точність: {acuracy}\n mse: {mse}\n r2: {r2}')

найкращі параметри: {'max_depth': 5, 'n_estimators': 10}

точність: 0.5714285714285714
 mse: 0.42857142857142855
 r2: -0.7788235294117645


In [223]:
y_test_reset = y_test.reset_index(drop=True)

logReg2 = LogisticRegression(**logRegGrid.best_params_, max_iter=2000, solver='liblinear')
logReg2.fit(X_train, y_train)

np.random.seed(42)
randomIndices = np.random.choice(X_test.shape[0], 10, replace=False)
X_random = X_test[randomIndices]
y_random_actual = y_test_reset[randomIndices]

y_random_pred = logReg2.predict(X_random)

comparisonDataFrame = pd.DataFrame({
    'Sample Index': randomIndices,
    'Real Class': y_random_actual,
    'Predicted Class': y_random_pred
})

comparisonDataFrame

,Sample Index,Real Class,Predicted Class
73,73,1,0
0,0,0,0
58,58,1,0
22,22,1,0
12,12,1,0
70,70,0,1
10,10,0,0
18,18,0,0
4,4,1,0
65,65,0,0


Як можемо бачити, без даних про стать, ми не можемо створити, простими інструментами, модель, що могла хоч трохи спрогнозувати шанс виживання пасажиру

## Висновок

будь яка зі створених моделей, що враховує параметр статі пасажиру, покриває 100% передбаченнь випадків через 100% кореляцію параметру статі та шансу на виживання. Без цього параметру, створення моделі неможливе